In [ ]:
!pip install -U selenium
!pip install -U pandas
!pip install -U bs4

In [1]:
# Importing Packages:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time

import pandas as pd
from bs4 import BeautifulSoup

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

driver = webdriver.Chrome(options = options)

In [3]:
driver.get('https://www.renodepot.com/webapp/wcs/stores/servlet/CategorySearchDisplay?pageSize=infinite&catalogId=10551&top=Y&urlLangId=-1&categoryId=32504&langId=-1&storeId=10701&productCategory=14745&page=1')
driver.refresh()

In [4]:
selectMTL = driver.find_element_by_id("department")
selectMTL.click()

In [5]:
SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [6]:
page_source = driver.page_source

In [7]:
soup = BeautifulSoup(page_source, 'lxml')

In [8]:
products = soup.find_all('a', class_ = 'product_box')
make = []
makeTool = []
deltaPrice = []
normPrice = []
salePrice = []
productURL = []

for product in products:
    make.append(product.find('div', class_ = 'product_manufacturer').get_text().strip())
    makeTool.append(product.find('div', class_ = 'product_name').get_text().strip())
    salePrice.append(product.find('span', class_ = 'integer').get_text().strip())
    productURL.append(product.get('href'))  
    try:
        deltaPrice.append(product.find('div', class_ = 'rebate').get_text().strip())
        normPrice.append(product.find('span', class_ = 'linethrough').get_text().strip())
    except:
        deltaPrice.append('0')
        normPrice.append(product.find('span', class_ = 'integer').get_text().strip())
        continue
        
     

In [9]:
product_df = pd.DataFrame(columns=['Make', 'Tool_Type', 'Regular_Price', 'Sale_Price', 'Rebate', 'Product_URL'])

product_df['Make'] = make
product_df['Tool_Type'] = makeTool
product_df['Regular_Price'] = normPrice
product_df['Sale_Price'] = salePrice
product_df['Rebate'] = deltaPrice
product_df['Product_URL'] = productURL

In [10]:
product_df

,Make,Tool_Type,Regular_Price,Sale_Price,Rebate,Product_URL
0,METABO HPT,"Pneumatic Stripe Brad Nailer - 5/8""-2""",$179.00,89,Save $90.00,https://www.renodepot.com/en/hitachi-pneumatic...
1,MAKITA,Set of 2 Cordless Tools 18V,$319.00,189,Save $130.00,https://www.renodepot.com/en/set-of-2-cordless...
2,MAKITA,Set of 2 Cordless Tools - 18 V Lithium-ion,$399.00,249,Save $150.00,https://www.renodepot.com/en/set-of-2-cordless...
3,BOSCH,"Small Angle Grinder - 4 1/2"" - Set of 2",$149.00,99,Save $50.00,https://www.renodepot.com/en/small-angle-grind...
4,BOSCH,Laser Measuring Tool - 100 ft,$89.99,59,Save $30.00,https://www.renodepot.com/en/laser-measuring-t...
...,...,...,...,...,...,...
1051,FULLER,WAVEX HAMMER 24OZ,29,29,0,https://www.renodepot.com/en/wavex-hammer-24oz...
1052,FULLER,Wood chisel,14,14,0,https://www.renodepot.com/en/wood-chisel-1422113
1053,FULLER,"""Phillips"" Screwdriver",13,13,0,https://www.renodepot.com/en/phillips-screwdri...
1054,FULLER,"Steel Slip joint Pliers - 6"" - Black and Red",6,6,0,https://www.renodepot.com/en/fuller-steel-slip...
